In [1]:
from qiskit_nature.second_q.hamiltonians import FermiHubbardModel
from qiskit_nature.settings import QiskitNatureSettings
from qiskit_nature.second_q.hamiltonians.lattices import (
    BoundaryCondition,
    LineLattice,
    HyperCubicLattice,
    SquareLattice,
)


import qiskit.quantum_info
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import DensityMatrix

import numpy as np



from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.problems import LatticeModelProblem


import psutil
import numpy as np
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw
import time
import argparse
from qiskit_algorithms.optimizers import SLSQP, SPSA
from qiskit_algorithms.minimum_eigensolvers import VQE

from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.utils import algorithm_globals


In [3]:
def FHM(t, v, u, size):
    '''
    Second Quantized Hamiltonian for the Fermi-Hubbard model in a periodic cube
    Returns the hamiltonian
    :param self:
    :param t:the interaction parameter
    :param v:the onsite potential
    :param u:
    :param size: Size of the lattice
    :return:
    '''

    boundary_condition = (
        BoundaryCondition.OPEN,
    )*3
    linear_lattice = HyperCubicLattice(size=[size]*3, boundary_condition=boundary_condition)
    QiskitNatureSettings.use_pauli_sum_op = False

    fhm_linear = FermiHubbardModel(
        linear_lattice.uniform_parameters(
            uniform_interaction=t,
            uniform_onsite_potential=v,
        ),
        onsite_interaction=u,
    )
    ham = fhm_linear.second_q_op()
    
    return ham


In [6]:
from qiskit_nature.second_q.mappers import JordanWignerMapper
def FHMJW(t, v, u, size):
    '''
    Second Quantized Hamiltonian for the Fermi-Hubbard model in a periodic cube
    Returns the hamiltonian
    :param self:
    :param t:the interaction parameter
    :param v:the onsite potential
    :param u:
    :param size: Size of the lattice
    :return:
    '''

    boundary_condition = (
        BoundaryCondition.OPEN,
    )*3
    linear_lattice = HyperCubicLattice(size=[size]*3, boundary_condition=boundary_condition)
    QiskitNatureSettings.use_pauli_sum_op = False

    fhm_linear = FermiHubbardModel(
        linear_lattice.uniform_parameters(
            uniform_interaction=t,
            uniform_onsite_potential=v,
        ),
        onsite_interaction=u,
    )
    ham = fhm_linear.second_q_op()
    hamJW=JordanWignerMapper().map(ham)
    return hamJW
len(FHMJW(1, 1, 1, 4).paulis)

769

In [9]:
from functools import reduce
def construct_2Qhamiltonian_with_majorana(num_sites, hopping_parameter, onsite_interaction):
    ham = FHM(1, 1, 1, 4)
    hamJW=JordanWignerMapper().map(ham)
    num_spin_orbitals = ham.num_spin_orbitals
    tot=[]
    for label, coeff in ham.items():
        ops = label.split(" ")
        
        mult=[]
        for o in ops:
     
            typ, site = o.split("_")
            site = int(site)
            if typ == "+":
                majop = majorana_to_Pauli(site,num_spin_orbitals,True)-majorana_to_Pauli(site,num_spin_orbitals,False)
            elif typ=="-":
                majop = majorana_to_Pauli(site,num_spin_orbitals,True)+majorana_to_Pauli(site,num_spin_orbitals,False)
            mult.append(majop)
        majterm = reduce(lambda a,b: a@b,mult)
        tot.append(coeff*majterm)
    return reduce(lambda a,b:a+b,tot).simplify(),hamJW
def majorana_to_Pauli(j, num_spin_orbital,maj_site):
    if maj_site:
        
        return SparsePauliOp((num_spin_orbital-j-1)*"I"+"X"+"Z"*j,coeffs=[0.5])
    else:
        return SparsePauliOp((num_spin_orbital-j-1)*"I"+"Y"+"Z"*j,coeffs=[0.5j])

In [10]:
h1,h2=construct_2Qhamiltonian_with_majorana(1,1,1)
h1.size,h2.size

(769, 769)